In [2]:
import pandas as pd
import numpy as np
import glob
#import nibabel as nib
from scipy.ndimage import label
from numpy.linalg import inv

## msid + mseid data

In [3]:
#Construct data table
final_cols = ["msid",
             "tot_years_gt1_lesion_present",
             "inf_years_gt1_lesion_present",
             "jux_years_gt1_lesion_present",
             "per_years_gt1_lesion_present",
             "sub_years_gt1_lesion_present",
             "tot_lesion_presentation",
             "inf_lesion_presentation",
             "jux_lesion_presentation",
             "per_lesion_presentation",
             "sub_lesion_presentation",
             "comparison",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "tot_lesion_vol_change_percent",
             "tot_lesion_vol_change_absolute",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "inf_lesion_vol_change_percent",
             "inf_lesion_vol_change_absolute",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "jux_lesion_vol_change_percent",
             "jux_lesion_vol_change_absolute",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "per_lesion_vol_change_percent",
             "per_lesion_vol_change_absolute",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute",
             "sub_lesion_vol_change_percent",
             "sub_lesion_vol_change_absolute",
             "tot_number_increasing_0",
             "tot_number_increasing_10",
             "tot_number_increasing_20",
             "tot_number_increasing_50",
             "tot_number_increasing_100",
             "inf_number_increasing_0",
             "inf_number_increasing_10",
             "inf_number_increasing_20",
             "inf_number_increasing_50",
             "inf_number_increasing_100",
             "jux_number_increasing_0",
             "jux_number_increasing_10",
             "jux_number_increasing_20",
             "jux_number_increasing_50",
             "jux_number_increasing_100",
             "per_number_increasing_0",
             "per_number_increasing_10",
             "per_number_increasing_20",
             "per_number_increasing_50",
             "per_number_increasing_100",
             "sub_number_increasing_0",
             "sub_number_increasing_10",
             "sub_number_increasing_20",
             "sub_number_increasing_50",
             "sub_number_increasing_100",
             "tot_number_decreasing_0",
             "tot_number_decreasing_10",
             "tot_number_decreasing_20",
             "tot_number_decreasing_50",
             "tot_number_decreasing_100",
             "inf_number_decreasing_0",
             "inf_number_decreasing_10",
             "inf_number_decreasing_20",
             "inf_number_decreasing_50",
             "inf_number_decreasing_100",
             "jux_number_decreasing_0",
             "jux_number_decreasing_10",
             "jux_number_decreasing_20",
             "jux_number_decreasing_50",
             "jux_number_decreasing_100",
             "per_number_decreasing_0",
             "per_number_decreasing_10",
             "per_number_decreasing_20",
             "per_number_decreasing_50",
             "per_number_decreasing_100",
             "sub_number_decreasing_0",
             "sub_number_decreasing_10",
             "sub_number_decreasing_20",
             "sub_number_decreasing_50",
             "sub_number_decreasing_100",
             "tot_new_lesions",
             "tot_lost_lesions",
             "inf_new_lesions",
             "inf_lost_lesions",
             "jux_new_lesions",
             "jux_lost_lesions",
             "per_new_lesions",
             "per_lost_lesions",
             "sub_new_lesions",
             "sub_lost_lesions",
             "lesion",
             "in_both",
             "overlap",
             "volume_change",
             "change_in_lesion_type"]
analysis = {}
for item in final_cols:
    analysis[item] = []
prev_data = pd.DataFrame.from_csv('clinical_info.csv')
prev_data.columns

Index([u'mse', u'tot_total_lesions', u'tot_max_lesion_volume',
       u'tot_min_lesion_volume', u'tot_total_volume_of_lesions',
       u'tot_percent_volume_of_lesions', u'tot_average_volume',
       u'tot_median_volume', u'inf_total_lesions', u'inf_max_lesion_volume',
       u'inf_min_lesion_volume', u'inf_total_volume_of_lesions',
       u'inf_percent_volume_of_lesions', u'inf_average_volume',
       u'inf_median_volume', u'jux_total_lesions', u'jux_max_lesion_volume',
       u'jux_min_lesion_volume', u'jux_total_volume_of_lesions',
       u'jux_percent_volume_of_lesions', u'jux_average_volume',
       u'jux_median_volume', u'per_total_lesions', u'per_max_lesion_volume',
       u'per_min_lesion_volume', u'per_total_volume_of_lesions',
       u'per_percent_volume_of_lesions', u'per_average_volume',
       u'per_median_volume', u'sub_total_lesions', u'sub_max_lesion_volume',
       u'sub_min_lesion_volume', u'sub_total_volume_of_lesions',
       u'sub_percent_volume_of_lesions', u'sub_a

In [10]:
#created matrix of tps
tp_matrix = {}
count = 1
for x in range(1,10):
    for y in range(x+1,11):
        tp_matrix["%s,%s" %(x,y)]=count
        count += 1
tp_matrix

#create sorted DataFrame in order of msid and tp
#create list of msids for looping purposes
df = pd.DataFrame(prev_data).sort_values(by=['msid','tp']).reset_index()
cols = pd.Index(df.columns)
msids = df.msid.values
types = ['tot','inf','jux','per','sub']

#percent change function
def percentchange(num2,num1):
    if num1 == 0 or num2 == 0:
        value = 0
    else:
        value = ((float(num2)-float(num1))/(float(num1))) * 100
    return round(value,2)

#lesion presentation pattern determination function
def pattern(lesionlist):
    truth = []
    for x in lesionlist:
        if x >= 1:
            truth.append(True)
        else:
            truth.append(False)
    if truth.count(True) == len(lesionlist):
        return "throughout"
    elif truth.count(False) == len(lesionlist):
        return "never"
    else:
        if truth[0] == False:
            return "late"
        elif truth[-1] == False:
            return "early"
        else: 
            return "intermittent"

#meat of the program
for ids in msids:
    #collect values per tp per msid 
    #specifically collect mseids, total lesion count, total lesion volume, total lesion count (type), total lesion volume (type)
    msid_vals = {}
    current_msid = df['msid']==ids
    tp_list = df[current_msid]['tp'].values 
    mseid_list = df[current_msid]['mse'].values
    for les in types:
        msid_vals['%s_tot_list' % les] = df[current_msid]['%s_total_lesions' % les].values
        msid_vals['%s_vol_list' % les] = df[current_msid]['%s_total_volume_of_lesions' % les].values    
    
    #calculate number of year by year comparisons needed for each msid
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(yrx+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])
            
    #calculate totals, changes for each lesion type and entire brain
    for les in types:
        msid_vals['%s_tot_list' % les] = df[current_msid]['%s_total_lesions' % les].values
        print msid_vals['%s_tot_list' % les],
        msid_vals['%s_change_list' % les] = []
        for comp in range(len(tp_ix_comp)):
            les_change_abs = msid_vals['%s_tot_list' % les][tp_ix_comp[comp][1]] - msid_vals['%s_tot_list' % les][tp_ix_comp[comp][0]]
            les_change_per = percentchange(msid_vals['%s_tot_list' % les][tp_ix_comp[comp][1]],msid_vals['%s_tot_list' % les][tp_ix_comp[comp][0]])
            msid_vals['%s_change_list' % types[les]].append([les_change_abs,les_change_per])
    print        
    #append calculations to dictionary for dataframe creation
    for count in range(len(msid_vals['tot_change_list'])):
        for les in types:
            #append absolute and percent change statistics
            analysis["%s_lesion_change_percent" % les].append(msid_vals['%s_change_list' % les][count][1])
            analysis["%s_lesion_change_absolute" % les].append(msid_vals['%s_change_list' % les][count][0])
            
            #append counts for years of comparison
            total1 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][0]]
            total2 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][1]]
            analysis["%s_lesion_counts" % les].append([total1,total2])
            
            #find out and append how many years greater than 1 lesion present for each lesion type + total lesions
            xcount = 0
            for les_count in range(len(msid_vals['%s_tot_list' % les])):
                if msid_vals['%s_tot_list' % les][les_count] == 0:
                    xcount += 1
            analysis["%s_years_gt1_lesion_present" % les].append(count)
            
            #determine presentation pattern (throughout, never, came late, left early, intermittent)
            analysis["%s_lesion_presentation" % les].append(pattern(msid_vals['%s_tot_list' % les]))
        
        #append mkid and years of comparison
        analysis["msid"].append(testids[x])
        analysis["comparison"].append(tp_yr_comp[count])
    
    #print values for debugging purposes
change_analysis = pd.DataFrame(analysis,columns=final_cols)
analysis

[11] [0] [0] [1] [10]
[2] [1] [0] [0] [1]
[14] [1] [3] [4] [6]
[14] [1] [4] [5] [4]
[37] [0] [11] [10] [16]
[19] [1] [2] [9] [7]
[4] [0] [1] [2] [1]
[19] [0] [4] [11] [4]
[25] [0] [6] [3] [16]
[15] [0] [2] [8] [5]
[38] [1] [9] [12] [16]
[4] [0] [0] [2] [2]
[11] [0] [2] [4] [5]
[15] [1] [6] [4] [4]
[6] [1] [0] [3] [2]
[17] [1] [4] [9] [3]
[9] [0] [3] [3] [3]
[71] [2] [26] [16] [27]
[4] [1] [1] [0] [2]
[20] [2] [3] [9] [6]
[15] [0] [1] [7] [7]
[3] [0] [0] [1] [2]
[4] [0] [1] [1] [2]
[3] [0] [0] [0] [3]
[12] [1] [2] [4] [5]
[12] [1] [2] [5] [4]
[36] [0] [4] [13] [19]
[9] [1] [4] [3] [1]
[23] [0] [17] [3] [3]
[17] [3] [2] [7] [5]
[7] [0] [2] [2] [3]
[15] [1] [3] [3] [8]
[26] [1] [9] [7] [9]
[34] [3] [10] [13] [8]
[81] [6] [46] [6] [23]
[18] [0] [4] [9] [5]
[38] [2] [11] [7] [18]
[45] [3] [19] [9] [14]
[41] [4] [14] [7] [16]
[37] [2] [8] [21] [6]
[18] [0] [8] [5] [5]
[11] [0] [5] [4] [2]
[8] [0] [0] [3] [5]
[5] [0] [1] [3] [1]
[34] [1] [7] [3] [23]
[7] [0] [4] [0] [3]
[39] [0] [19] [1] [19]

{'change_in_lesion_type': [],
 'comparison': [],
 'in_both': [],
 'inf_lesion_change_absolute': [],
 'inf_lesion_change_percent': [],
 'inf_lesion_presentation': [],
 'inf_lesion_vol_change_absolute': [],
 'inf_lesion_vol_change_percent': [],
 'inf_lost_lesions': [],
 'inf_new_lesions': [],
 'inf_number_decreasing_0': [],
 'inf_number_decreasing_10': [],
 'inf_number_decreasing_100': [],
 'inf_number_decreasing_20': [],
 'inf_number_decreasing_50': [],
 'inf_number_increasing_0': [],
 'inf_number_increasing_10': [],
 'inf_number_increasing_100': [],
 'inf_number_increasing_20': [],
 'inf_number_increasing_50': [],
 'inf_years_gt1_lesion_present': [],
 'jux_lesion_change_absolute': [],
 'jux_lesion_change_percent': [],
 'jux_lesion_presentation': [],
 'jux_lesion_vol_change_absolute': [],
 'jux_lesion_vol_change_percent': [],
 'jux_lost_lesions': [],
 'jux_new_lesions': [],
 'jux_number_decreasing_0': [],
 'jux_number_decreasing_10': [],
 'jux_number_decreasing_100': [],
 'jux_number_de

## Dummy Code

In [45]:
#created test data and dataframe, sorted by mkid and tp
test = {"mkid" : ["mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4"],
        "mkeid" : ["mke009","mke002","mke003","mke004","mke005","mke006","mke007","mke008",
                   "mke001","mke010","mke011","mke012"],
        "tp" : [1,1,1,1,2,2,2,3,3,3,5,4],
        "tot_total_lesions" : [5,10,6,14,1,11,24,13,25,27,10,4],
        "sub_total_lesions" : [1,5,0,5,1,0,11,3,12,9,3,2],
        "inf_total_lesions" : [1,2,2,3,0,1,4,3,25,4,3,0],
        "per_total_lesions" : [2,2,4,4,0,5,10,3,4,10,3,1],
        "jux_total_lesions" : [1,1,0,4,0,4,0,4,5,4,1,1],
       }
testcolumns = test.keys()
testdf = pd.DataFrame(test, columns=[testcolumns]).sort_values(by=['mkid','tp'])
testids = testdf.mkid.values
types = ['tot','inf','jux','per','sub']


#analysis dataframe
analysiscol=["mkid",
             "tot_years_gt1_lesion_present",
             "inf_years_gt1_lesion_present",
             "jux_years_gt1_lesion_present",
             "per_years_gt1_lesion_present",
             "sub_years_gt1_lesion_present",
             "tot_lesion_presentation",
             "inf_lesion_presentation",
             "jux_lesion_presentation",
             "per_lesion_presentation",
             "sub_lesion_presentation",
             "comparison",
             "tot_lesion_counts",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "inf_lesion_counts",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "jux_lesion_counts",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "per_lesion_counts",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "sub_lesion_counts",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute"]
analysis = {}
for item in analysiscol:
    analysis[item] = []

#lesion presentation pattern determination function
def pattern(lesionlist):
    truth = []
    for x in lesionlist:
        if x >= 1:
            truth.append(True)
        else:
            truth.append(False)
    if truth.count(True) == len(lesionlist):
        return "throughout"
    elif truth.count(False) == len(lesionlist):
        return "never"
    else:
        if truth[0] == False:
            return "late"
        elif truth[-1] == False:
            return "early"
        else: 
            return "intermittent"
        
#perform calculations
print "mkid", "comparison", "abs_lesion_change", "per_lesion_change"
for x in range(0,len(testids),3):
    #collect values mkid,mkeid,tp
    msid_vals = {}
    val = testdf['mkid']==testids[x] #shortcut to gathering variables by mkid
    tp_list = testdf[val]['tp'].values 
    mkeid_list = testdf[val]['mkeid'].values

    #collect tp yrs and ixs for change analysis
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(yrx+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])

    #calculate totals, changes for each lesion type and entire brain
    for les in range(len(types)):
        msid_vals['%s_tot_list' % types[les]] = testdf[val]['%s_total_lesions' % types[les]].values
        msid_vals['%s_change_list' % types[les]] = []
        for comp in range(len(tp_ix_comp)):
            les_change_abs = msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]] - msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]]
            les_change_per = percentchange(msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]],msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]])
            msid_vals['%s_change_list' % types[les]].append([les_change_abs,les_change_per])

    #append calculations to dictionary for dataframe creation
    for count in range(len(msid_vals['tot_change_list'])):
        for les in types:
            #append absolute and percent change statistics
            analysis["%s_lesion_change_percent" % les].append(msid_vals['%s_change_list' % les][count][1])
            analysis["%s_lesion_change_absolute" % les].append(msid_vals['%s_change_list' % les][count][0])
            
            #append counts for years of comparison
            total1 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][0]]
            total2 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][1]]
            analysis["%s_lesion_counts" % les].append([total1,total2])
            
            #find out and append how many years greater than 1 lesion present for each lesion type + total lesions
            xcount = 0
            for les_count in range(len(msid_vals['%s_tot_list' % les])):
                if msid_vals['%s_tot_list' % les][les_count] == 0:
                    xcount += 1
            analysis["%s_years_gt1_lesion_present" % les].append(xcount)
            
            #determine presentation pattern (throughout, never, came late, left early, intermittent)
            analysis["%s_lesion_presentation" % les].append(pattern(msid_vals['%s_tot_list' % les]))
        
        #append mkid and years of comparison
        analysis["mkid"].append(testids[x])
        analysis["comparison"].append(tp_yr_comp[count])

lulz = pd.DataFrame(analysis,columns=analysiscol)
lulz

mkid comparison abs_lesion_change per_lesion_change


,mkid,tot_years_gt1_lesion_present,inf_years_gt1_lesion_present,jux_years_gt1_lesion_present,per_years_gt1_lesion_present,sub_years_gt1_lesion_present,tot_lesion_presentation,inf_lesion_presentation,jux_lesion_presentation,per_lesion_presentation,...,inf_lesion_change_absolute,jux_lesion_counts,jux_lesion_change_percent,jux_lesion_change_absolute,per_lesion_counts,per_lesion_change_percent,per_lesion_change_absolute,sub_lesion_counts,sub_lesion_change_percent,sub_lesion_change_absolute
0,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,-1,"[1, 0]",0.0,-1,"[2, 0]",0.00,-2,"[1, 1]",0.00,0
1,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,24,"[1, 5]",400.0,4,"[2, 4]",100.00,2,"[1, 12]",1100.00,11
2,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,25,"[0, 5]",0.0,5,"[0, 4]",0.00,4,"[1, 12]",1100.00,11
3,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,-1,"[1, 4]",300.0,3,"[2, 5]",150.00,3,"[5, 0]",0.00,-5
4,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,2,"[1, 4]",300.0,3,"[2, 10]",400.00,8,"[5, 9]",80.00,4
5,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,3,"[4, 4]",0.0,0,"[5, 10]",100.00,5,"[0, 9]",0.00,9
6,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,2,"[0, 0]",0.0,0,"[4, 10]",150.00,6,"[0, 11]",0.00,11
7,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,1,"[0, 1]",0.0,1,"[4, 3]",-25.00,-1,"[0, 3]",0.00,3
8,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,-1,"[0, 1]",0.0,1,"[10, 3]",-70.00,-7,"[11, 3]",-72.73,-8
9,mk4,0,1,0,0,0,throughout,early,throughout,throughout,...,0,"[4, 4]",0.0,0,"[4, 3]",-25.00,-1,"[5, 3]",-40.00,-2
